# US EPA Air Quality System API Example
This example illustrates how to request data from the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. This is a historical API and does not provide real-time air quality data. The documentation for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

This notebook works systematically through example calls, requesting an API key, using 'list' to get various IDs and parameter values, and using 'daily summary' to get summary data that meets specific condistions. The notebook contains example function definitions that could be reused in other code. In general, the notebook explains each step along the way, referring back to possible output. Some of the explanations are tailored to the specific example requests of the API. Changing values to explore the results of the API is probably useful, but that will result in some explanations being out of sync with the outputs.

The US EPA was created in the early 1970's. The EPA reports that they only started broad based monitoring with standardized quality assurance procedures in the 1980's. Many counties will have data starting somewhere between 1983 and 1988. However, some counties still do not have any air quality monitoring stations. The API helps resolve this by providing calls to search for monitoring stations and data using either station ids, or a county designation or a geographic bounding box. This example code provides examples of the county based and bounding box based API calls. Some additional information on the Air Quality System can be found in the EPA FAQ on the system.

The end goal of this example is to get to some values that we might use for the Air Quality Index or AQI. You might see this reported on the news, most often around smog, but more frequently with regard to smoke. The AQI index is meant to tell us something about how healthy or clean the air is on any day. The AQI is actually a somewhat complext measure. When I started this example I looked up how to calculate the AQI so that I would know roughly what goes into that value.

To get a key in order to execute the below code you can visit [this](https://aqs.epa.gov/aqsweb/documents/data_api.html#signup) website and follow the instructions.

## License
This code example is taken from the code which was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the Creative Commons CC-BY license. Revision 1.1 - September 5, 2023

In [1]:
#
#    These are standard python modules
#
#import json, time, urllib.parse
import json, time
#
#    The 'requests' module is a distribution module for making web requests.
#
import requests
from tqdm import tqdm
import pandas as pd

Before you can use the API you need to request a key. You will use an email address to make the request. The EPA then sends a confirmation email link and a 'key' that you use for all other requests.

You only need to sign-up once, unless you want to invalidate your current key (by getting a new key) or you lose your key.

In [2]:
#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
#
AQS_REQUEST_TEMPLATE = {
    "email":      "",
    "key":        "",
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}

USERNAME = "<YOUR_EMAIL>"
APIKEY = "<YOUR_API_KEY>"

Once you have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. The one problem here is that some of the responses rely on jargon or terms-of-art. That is, one needs to know a bit about the way atmospheric sciece works to understand some of the terms. ... Good thing we can use the web to search for terms we don't know!

In [3]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL,
                      endpoint_action = API_ACTION_LIST_CLASSES,
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    """
    Request information about available data classes using the Air Quality System API.

    This function sends a request to the Air Quality System API to retrieve a list of available data classes.
    Data classes are categories of air quality data, and this request provides information about what data is
    available for retrieval.

    Args:
        email_address (str): Your email address for API access.
        key (str): Your API key for authentication.
        endpoint_url (str): The base URL for the API (default is API_REQUEST_URL).
        endpoint_action (str): The specific API action for requesting data class information
            (default is API_ACTION_LIST_CLASSES).
        request_template (dict): A dictionary containing request parameters.
            Used to set up the request (default is AQS_REQUEST_TEMPLATE).
        headers (dict): Optional headers to include in the HTTP request.

    Returns:
        dict or None: A dictionary containing information about available data classes retrieved from the API, or None if the request fails.

    Raises:
        Exception: If either email_address or key is missing in the request template.

    Example:
        To retrieve information about available data classes, you can use this function as follows:
        ```
        data_info = request_list_info(email_address='your_email@example.com', key='your_api_key')
        if data_info:
            print(data_info)
        else:
            print("Request failed.")
        ```
    """
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key

    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [4]:
#
#   The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clustors of
#   sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the
#   a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from
#   that specific sensor.
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

We're interested in getting to something that might be the Air Quality Index (AQI). You see this reported on the news - often around smog values, but also when there is smoke in the sky. The AQI is a complex measure of different gasses and of the particles in the air (dust, dirt, ash ...).

From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.

In [5]:
#
#   Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group)
#   The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
#   We'll use that to make another list request.
#
AQI_PARAM_CLASS = "AQI POLLUTANTS"

In [6]:
#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We should now have (above) a response containing a set of sensor ID numbers. The list should include the sensor numbers as well as a description or name for each sensor.

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up.

What I did below was to break the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [7]:
#
#   Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
#   It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
#   all seven of the code types. We can only have a max of 5 sensors/values request per param.
#
#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"
#
#

Air quality monitoring stations are located all over the US at different locations. We will need some sample locations to experiment with different locations to see what kinds of values come back from different sensor requests.

This list includes the FIPS number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.

Just two example cities to explore.

In [8]:
#
#   We'll use these two city locations in the examples below.
#
CITY_LOCATIONS = {
    'Redding' :       {'city'   : 'Redding',
                       'county' : 'Shasta',
                       'state'  : 'California',
                       'fips'   : '06089',
                       'latlon' : [40.5865, -122.3916] }
}



Given our CITY_LOCATIONS constant we can now find which monitoring locations are nearby. One option is to use the county to define the area we're interest in. You can get the EPA to list their monitoring stations by county. You can also get a set of monitoring stations by using a bounding box of latitude, longitude points. For this example, we'll use the county approach. There is a bounding box example later in this notebook.

In [9]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city selected from the CITY_LOCATIONS dictionary
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['Redding']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['Redding']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": "Redding - Health Department"
    },
    {
        "code": "0005",
        "value_represented": null
    },
    {
        "code": "0006",
        "value_represented": null
    },
    {
        "code": "0007",
        "value_represented": "Anderson - North Street"
    },
    {
        "code": "0008",
        "value_represented": "Shasta Lake-La Mesa"
    },
    {
        "code": "0009",
        "value_represented": "Shasta Lake - Lake Blvd"
    },
    {
        "code": "1001",
        "value_represented": null
    },
    {
        "code": "1002",
        "value_represented": null
    },
    {
        "code": "2001",
        "value_represented": null
    },
    {
        "code": "2002",
        "value_represented": null
    },
    {
        "code": "3001",
        "value_represented": nul

## Making a daily summary request
The function below is designed to encapsulate requests to the EPA AQS API. When calling the function one should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call simply pass in the parameters that need to change, like date ranges.

Another function below provides an example of extracting values and restructuring the response to make it a little more usable.

In [10]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date.
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL,
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY,
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    """
    Make a request to obtain daily summary data for a specific set of parameters and time window.

    Args:
        email_address (str): Email address for API access.
        key (str): API key for authentication.
        param (str): Comma-separated list of parameter codes to request.
        begin_date (str): Start date for the data request (YYYYMMDD).
        end_date (str): End date for the data request (YYYYMMDD).
        fips (str): FIPS code for state and county. Must be 5 characters (e.g., '06089' for Shasta County, California).
        endpoint_url (str): URL for the API endpoint.
        endpoint_action (str): Specific action to perform for the request.
        request_template (dict): Template for request parameters.
        headers (dict): Additional headers for the request.

    Returns:
        dict: JSON response containing daily summary data.
    """

    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']:
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']:
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']:
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']:
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)

    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        # if API_THROTTLE_WAIT > 0.0:
        #     time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

#
#    This is a list of field names - data - that will be extracted from each record
#
EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']


#
#    The function creates a summary record
def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
    """
    Extract and structure summary data from the API response.

    Args:
        r (dict): JSON response data from the API request.
        fields (list): List of field names to extract from each record.

    Returns:
        dict: Structured summary data organized by monitoring site, parameter, and date.
    """
    ## the result will be structured around monitoring site, parameter, and then date
    result = dict()
    data = r["Data"]
    for record in data:
        # make sure the record is set up
        site = record['site_number']
        param = record['parameter_code']
        #date = record['date_local']    # this version keeps the respnse value YYYY-
        date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
        if site not in result:
            result[site] = dict()
            result[site]['local_site_name'] = record['local_site_name']
            result[site]['site_address'] = record['site_address']
            result[site]['state'] = record['state']
            result[site]['county'] = record['county']
            result[site]['city'] = record['city']
            result[site]['pollutant_type'] = dict()
        if param not in result[site]['pollutant_type']:
            result[site]['pollutant_type'][param] = dict()
            result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
            result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
            result[site]['pollutant_type'][param]['method'] = record['method']
            result[site]['pollutant_type'][param]['data'] = dict()
        if date not in result[site]['pollutant_type'][param]['data']:
            result[site]['pollutant_type'][param]['data'][date] = list()

        # now extract the specified fields
        extract = dict()
        for k in fields:
            if str(k) in record:
                extract[str(k)] = record[k]
            else:
                # this makes sure we always have the requested fields, even if
                # we have a missing value for a given day/month
                extract[str(k)] = None

        # add this extraction to the list for the day
        result[site]['pollutant_type'][param]['data'][date].append(extract)

    return result

In [23]:
def process_pollutant_data(data):
    """
    Process raw pollutant data and calculate the daily average AQI.

    Args:
        data (dict): Raw pollutant data in a nested dictionary format.

    Returns:
        pandas.DataFrame: DataFrame containing Date, Pollutant Name, and Daily Average AQI.
    """
    records = []

    for station_id, station_data in data.items():
        for pollutant_id, pollutant_data in station_data['pollutant_type'].items():
            pollutant_name = pollutant_data['parameter_name']
            pollutant_data = pollutant_data['data']

            for date, date_data in pollutant_data.items():
                for entry in date_data:
                    aqi = entry['aqi']
                    if aqi is not None:
                        records.append((date, pollutant_name, aqi))

    df = pd.DataFrame(records, columns=['Date', 'Pollutant Name', 'Avg AQI'])
    daily_avg_aqi = df.groupby(['Date', 'Pollutant Name'])['Avg AQI'].mean().reset_index()

    return daily_avg_aqi

In [24]:
# Define the columns
columns = ['Date', 'Pollutant Name', 'Avg AQI']

# Create empty DataFrames with the specified columns
gas_data = pd.DataFrame(columns=columns)
particulate_data = pd.DataFrame(columns=columns)

In [13]:
# Iterate through a range of years
for year in tqdm(range(1963, 2024), position=0, leave=True, desc="Processing Years"):
    # Define the start and end dates for the current year
    begin_date = str(year) + "0101"
    end_date = str(year) + "1231"

    # Prepare a request template for gaseous pollutants
    request_data = AQS_REQUEST_TEMPLATE.copy()
    request_data['email'] = USERNAME
    request_data['key'] = APIKEY
    request_data['param'] = AQI_PARAMS_GASEOUS
    request_data['state'] = CITY_LOCATIONS['Redding']['fips'][:2]
    request_data['county'] = CITY_LOCATIONS['Redding']['fips'][2:]

    # Request daily summary data for gaseous pollutants
    gaseous_aqi = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)

    # Extract and process the gaseous pollutant data
    extract_gaseous = extract_summary_from_response(gaseous_aqi)

    # Update the request template for particulate pollutants
    request_data['param'] = AQI_PARAMS_PARTICULATES

    # Request daily summary data for particulate pollutants
    particulate_aqi = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)

    # Extract and process the particulate pollutant data
    extract_particulate = extract_summary_from_response(particulate_aqi)

    # Convert the data into dataframes
    gaseous_df = process_pollutant_data(extract_gaseous)
    particulate_df = process_pollutant_data(extract_particulate)

    # Concatenate the dataframes
    gas_data = pd.concat([gas_data, gaseous_df])
    particulate_data = pd.concat([particulate_data, particulate_df])


Processing Years: 100%|██████████| 61/61 [01:58<00:00,  1.95s/it]


The form of the daily summary response is a bit verbose with lots of repeated values. What we'll do is create a data structure that relies on a hierarchical context to summarize the data.

The two responses (for Bend, OR) show that not every monitoring site produces values. As well, it looks like the monitoring sites only produce values for particulates and not for gaseous pollutants.

The next function takes the response and a set of fields that should be extracted for their data values. The code assumes those fields are available. If there are missing values something could certainly go wrong. The function creates a summary for each monitoring site.

In [18]:
combined_data = pd.concat([gas_data, particulate_data], ignore_index=True)
combined_data

,Date,Pollutant Name,Avg AQI
0,19720101,Carbon monoxide,24.0
1,19720101,Nitrogen dioxide (NO2),19.0
2,19720102,Carbon monoxide,23.0
3,19720102,Nitrogen dioxide (NO2),19.0
4,19720103,Carbon monoxide,23.0
...,...,...,...
30192,20230816,PM10 Total 0-10um STP,22.0
30193,20230816,PM2.5 - Local Conditions,40.0
30194,20230822,PM10 Total 0-10um STP,19.0
30195,20230828,PM10 Total 0-10um STP,21.0


In [26]:
# Convert the 'Date' column to a datetime format
combined_data['Date'] = pd.to_datetime(combined_data['Date'], format='%Y%m%d')

# Sort the data by 'Date' in ascending order and 'Avg AQI' in descending order
combined_data = combined_data.sort_values(['Date', 'Avg AQI'], ascending=[True, False])

# Group the data by 'Date' and keep the first entry (max AQI) for each date
combined_data = combined_data.groupby('Date').first().reset_index()

# Extract the 'Year' from the 'Date' column
combined_data['Year'] = combined_data['Date'].dt.year

# Group the data by 'Year' and select the top 10 AQI values for each year
top_10_aqi = combined_data.groupby('Year').apply(lambda x: x.nlargest(10, 'Avg AQI')).reset_index(drop=True)

# Calculate the mean of the top 10 AQI values for each year and rename the column
std_aqi = top_10_aqi.groupby('Year')['Avg AQI'].mean().reset_index()
std_aqi = std_aqi.rename(columns={'Avg AQI': 'Calculated AQI'})

In [28]:
# Convert the DataFrame to a JSON object
std_aqi_json = std_aqi.to_json(orient='records')

# Specify the path to the output file
output_file_path = 'std_aqi.json'

# Write the JSON data to the file
with open(output_file_path, 'w') as json_file:
    json_file.write(std_aqi_json)

# Close the file
json_file.close()
